### Determine equilibrium constants by optimization function
input: 
- measured absorbance (real data)
- epsilon of BPB (from pH 3, 3.75, 5.2; best fit)

output:
- K1 and K2 
- molar fraction of each species 

In [14]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from scipy.optimize import minimize 
import statistics 
%matplotlib inline
mpl.rcParams['figure.dpi']= 200

In [16]:
#Import Abs data
df=pd.read_csv(r'G:\Shared drives\Electrochemistry\04.Data\02.Processed\01.UVVis\20210225_rainbow_absorption\20210304_Abs_measured_pH_4.07.txt', sep="\t", header=None)
wavelength0=df[0].tolist()
A0=df[1].tolist()

In [17]:
#Import eps of BPB 
df2=pd.read_csv(r'G:\Shared drives\Electrochemistry\04.Data\01.Raw\01.UVVIS\20210225_rainbow_absorption\OutputDataframe\eps_from_pH3_3.75_5.2.txt', sep=",", header=None)
#print(df2.head(5))
eps_H2A0=df2[1].tolist()
eps_HAm0=df2[2].tolist()
eps_A2m0=df2[3].tolist()

In [20]:
#take wavelength from 400nm (i=1052) to 650nm (i=2321)
r=2321-1052
wavelength=[]
A=[]
eps_H2A=[]
eps_HAm=[]
eps_A2m=[]

for i in range(r):
    wavelength.append(wavelength0[1052+i])
    A.append(A0[1052+i])
    eps_H2A.append(eps_H2A0[1052+i])
    eps_HAm.append(eps_HAm0[1052+i])
    eps_A2m.append(eps_A2m0[1052+i])

In [7]:
#Objective function
def objective(x):
    return (A[wv]-(eps_H2A[wv]*x[0]+eps_HAm[wv]*x[1]+eps_A2m[wv]*x[2]))**2

In [8]:
#First guess based on pH
#Enter pH
pH = [4.07]
K1_0=10**(-3)
K2_0=10**(-4.6)

def mf_calc(pH, K1_0, K2_0):
    c_H = 10 **(-1*pH)
    denominator = c_H**2 + c_H*K1_0 + K1_0*K2_0
    return [c_H**2/denominator , c_H*K1_0/denominator , K1_0*K2_0/denominator]

x0 = []
for i in pH:
    x0.append(mf_calc(i, K1_0, K2_0))
print(x0)

[[0.06166618053211753, 0.7245144472964454, 0.213819372171437]]


In [9]:
#Constraints and bounds of the objective function
#bounds: +/- 20% of the initial guess
b1=[0,0.2]
b2=[0.5,0.9]
b3=[0,0.4]
bnds = [b1,b2,b3]

#constraint1 x1+x2+x3=1
def constraint1(x):
    return x[0]+x[1]+x[2]-1

#constraint2 x1+x2+x3>0
def constraint2(x):
    return x[0]+x[1]+x[2]

con1={'type':'eq', 'fun':constraint1}
con2={'type':'ineq', 'fun':constraint2}
cons=[con1,con2]

In [10]:
sol=[]
for wv in range(len(wavelength)):
    sol.append(minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons))
#print(sol[:5])

In [ ]:
#Calculate A predicted for every solution x 
A_p=[]
for i in range(len(sol)):
    sub=[]
    for wv in range(len(wavelength)):
        sub.append(eps_H2A[wv]*sol[i].x[0]+eps_HAm[wv]*sol[i].x[1]+eps_A2m[wv]*sol[i].x[2])
    A_p.append(sub)


In [10]:
#Compare A measured vs A predicted to find the best solution - error method
err=[]
for i in range(len(sol)):
    sub=[]
    for wv in range(len(wavelength)):
        sub.append((A[wv]-A_p[i][wv])**2)
    err.append(sub)

In [23]:
#mean of error and take the smalles one
mean_err=[]
for i in range(len(sol)):
    mean_err.append(statistics.mean(err[i]))

keep_err=mean_err[0]
for i in range(len(mean_err)):
    if (mean_err[i]<=keep_err):
        keep_err=mean_err[i]
        index_err=mean_err.index(keep_err)

In [26]:
#Results with error method
print(keep_err)
print(index_err)
print(mean_ratio[index_err])
print(std_ratio[index_err])
print(sol[index_err].x)
print(x0)

0.0011476543519662022
1080
1.0308243314211492
1.2124103040175698
[0.01952537 0.63942498 0.34104965]
[[0.06166618053211753, 0.7245144472964454, 0.213819372171437]]


In [ ]:
#Equilibrium constant with error method
K1=10**(-pH[0])*sol[index_err].x[1]/sol[index_err].x[0]
K2=10**(-pH[0])*sol[index_err].x[2]/sol[index_err].x[1]
pKa1=-math.log10(K1)
pKa2=-math.log10(K2)
print(pKa1)
print(pKa2)

In [12]:
#Compare A measured vs A predicted by taking the ratio
ratio=[]
for i in range(len(sol)):
    sub=[]
    for wv in range(len(wavelength)):
        sub.append(A[wv]/A_p[i][wv])
    ratio.append(sub)

In [13]:
#calculate mean and std of ratio 
mean_ratio =[]
std_ratio =[]
for i in range(len(sol)):
    mean_ratio.append(statistics.mean(ratio[i]))
    std_ratio.append(np.std(ratio[i]))

In [14]:
#solution with the smallest std
keep_ratio=std_ratio[0]
for i in range(len(sol)):
    if (std_ratio[i]<= keep_ratio):
        keep_ratio=std_ratio[i]
        index_ratio=std_ratio.index(keep_ratio)

In [27]:
#Results with ratio method and smallest std 
print(keep_ratio)
print(index_ratio)
print(mean_ratio[index_ratio])
print(std_ratio[index_ratio])
print(sol[index_ratio].x)
# print(x0)

0.13600393753208642
1239
1.063313390074601
0.13600393753208642
[0.04033804 0.7187778  0.24088416]


In [16]:
#Equilibrium constant with ratio method - smallest std 
K1=10**(-pH[0])*sol[index_ratio].x[1]/sol[index_ratio].x[0]
K2=10**(-pH[0])*sol[index_ratio].x[2]/sol[index_ratio].x[1]
pKa1=-math.log10(K1)
pKa2=-math.log10(K2)
print(pKa1)
print(pKa2)

2.81912014303553
4.544786418582098


In [22]:
#solution with the mean closest to 1
keep_mean=(mean_ratio[0]-1)**2
for i in range(len(sol)):
    delta=(mean_ratio[i]-1)**2
    if (delta <= keep_mean):
        keep_mean=delta
        index_m=mean_ratio.index(mean_ratio[i])

print(index_m)
print(mean_ratio[index_m])
print(std_ratio[index_m])
print(sol[index_m].x)

396
0.9999783002280536
0.6517320791325021
[0.01079139 0.67985021 0.3093584 ]


In [21]:
#Equilibrium constants with mean closest to 1
K1=10**(-pH[0])*sol[index_m].x[1]/sol[index_m].x[0]
K2=10**(-pH[0])*sol[index_m].x[2]/sol[index_m].x[1]
pKa1=-math.log10(K1)
pKa2=-math.log10(K2)
print(pKa1)
print(pKa2)

2.2706641020415486
4.411951318510545
